In [2]:
import requests
import re
import os
import time
from bs4 import BeautifulSoup

import base64
from io import BytesIO
from fontTools.ttLib import TTFont
import csv

In [3]:
def convert(getText):
    bs64Str = re.findall("charset=utf-8;base64,(.*?)\'\)", response.text)[0]
    binData = base64.decodebytes(bs64Str.encode())
    # 写入otf字体文件
    filePath01 = r'F:\技术分享\北京租房代码20201105\jiemi.otf'
    with open(filePath01, 'wb') as f:
        f.write(binData)
        f.close()
    # 解析字体库
    font01 = TTFont(filePath01)
    utfList = font01['cmap'].tables[0].ttFont.tables['cmap'].tables[0].cmap  
    # c = font.getBestCmap()
    retList = []
    for i in getText:
        # ord()以字符作为参数，返回对应的Unicode数值
        if ord(i) in utfList:
            text = int(utfList[ord(i)][-2:]) - 1
        else:
            text = i
        retList.append(text)
    return(''.join([str(j) for j in retList]).split('\n'))
s = requests.Session()
s.headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'accept-language': 'en-US,zh;q=0.8,zh-CN;q=0.7,zh-TW;q=0.5,zh-HK;q=0.3,en;q=0.2',
    'referer': 'https://bj.58.com/zufang/',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0',
}
s.get(url='https://bj.58.com/zufang/')
import time
for page in range(70):
    response=s.get('https://bj.58.com/zufang/pn{0}/?PGTID=0d300008-0000-1e0f-27d7-0238e53f2f24&ClickID=2'.format(page+1))
    print('第{}页'.format(page+1))
    soup=BeautifulSoup(response.text,'html.parser')
    strongbox=soup.find_all('li',class_='house-cell')+soup.find_all('li',class_='apartments')
    for box in strongbox:
        result=[]
        name=box.find_all('a',class_="strongbox")[0].text.replace('\n','').replace(' ','').replace('~','')
        room=box.find_all('p',class_='room')[0].text.replace('\n','').replace(' ','').replace('\xa0','')
        room=convert(room)[0]
        layout=room.split('卫')[0]+'卫'
        area=room.split('卫')[1]
        infor=box.find_all('p',class_='infor')[0].text.replace('\n','').replace(' ','').replace('  ','')
        money=box.find_all('div',class_='money')[0].text.replace('\n','')
        money=convert(money)[0]
        result=[name,layout,area,infor,money]
        with open('F:/技术分享/北京租房代码20201105/租房数据20201105.csv', 'a+',newline='', encoding='gb18030') as f:
            f_csv = csv.writer(f)
            f_csv.writerow(result)
    time.sleep(10)

第1页
第2页
第3页
第4页
第5页
第6页
第7页
第8页
第9页
第10页
第11页
第12页
第13页
第14页
第15页
第16页
第17页
第18页
第19页
第20页
第21页
第22页
第23页
第24页
第25页
第26页
第27页
第28页
第29页
第30页
第31页
第32页
第33页
第34页
第35页
第36页
第37页
第38页
第39页
第40页
第41页
第42页
第43页
第44页
第45页
第46页
第47页
第48页
第49页
第50页
第51页
第52页
第53页
第54页
第55页
第56页
第57页
第58页
第59页
第60页
第61页
第62页
第63页
第64页
第65页
第66页
第67页
第68页
第69页
第70页


In [39]:
import pandas as pd
fang=pd.read_csv('F:/技术分享/北京租房代码20201105/租房数据20201105.csv',encoding='gbk')
fang.head()

,租房信息,厅室,面积,详细地址,月租
0,昌平北七家未来科学城国瑞东二旗望都羊各庄新村出租,1室2厅1卫,77㎡,北七家羊各庄新村,2700元/月
1,首尔甜城林荫大道龒室龤厅龤卫龤鑶鸺麣元月南北通透,3室1厅1卫,98㎡,燕郊首尔甜城林荫大道,1850元/月
2,实拍双空套两个卧室都有门双卫中楼层东西通透随时起租,2室1厅2卫,38㎡,石门旭辉26街区距15号线南法信地铁站步行717m,3600元/月
3,二斜井惊现一居好户型快四总站交通方便,1室1厅1卫,41.67㎡,大峪城子二斜井,2400元/月
4,真实在租燕郊燕顺路首尔甜城全新家具家电豪装两居室,2室2厅1卫,86㎡,燕郊首尔甜城林荫大道,3000元/月


In [40]:
fang['室']=fang['厅室'].str.split('室',expand=True)[0]
fang['面积']=fang['面积'].str.split('㎡',expand=True)[0]
fang['月租']=fang['月租'].str.split('元/月',expand=True)[0]
fang.head()

,租房信息,厅室,面积,详细地址,月租,室
0,昌平北七家未来科学城国瑞东二旗望都羊各庄新村出租,1室2厅1卫,77,北七家羊各庄新村,2700,1
1,首尔甜城林荫大道龒室龤厅龤卫龤鑶鸺麣元月南北通透,3室1厅1卫,98,燕郊首尔甜城林荫大道,1850,3
2,实拍双空套两个卧室都有门双卫中楼层东西通透随时起租,2室1厅2卫,38,石门旭辉26街区距15号线南法信地铁站步行717m,3600,2
3,二斜井惊现一居好户型快四总站交通方便,1室1厅1卫,41.67,大峪城子二斜井,2400,1
4,真实在租燕郊燕顺路首尔甜城全新家具家电豪装两居室,2室2厅1卫,86,燕郊首尔甜城林荫大道,3000,2


In [41]:
fang['卫']=fang['厅室'].str.split('厅',expand=True)[1]
fang['卫']=fang['卫'].str.split('卫',expand=True)[1]
fang['厅']=fang['厅室'].str.split('卫',expand=True)[1]

fang.head()

,租房信息,厅室,面积,详细地址,月租,室,卫,厅
0,昌平北七家未来科学城国瑞东二旗望都羊各庄新村出租,1室2厅1卫,77,北七家羊各庄新村,2700,1,,
1,首尔甜城林荫大道龒室龤厅龤卫龤鑶鸺麣元月南北通透,3室1厅1卫,98,燕郊首尔甜城林荫大道,1850,3,,
2,实拍双空套两个卧室都有门双卫中楼层东西通透随时起租,2室1厅2卫,38,石门旭辉26街区距15号线南法信地铁站步行717m,3600,2,,
3,二斜井惊现一居好户型快四总站交通方便,1室1厅1卫,41.67,大峪城子二斜井,2400,1,,
4,真实在租燕郊燕顺路首尔甜城全新家具家电豪装两居室,2室2厅1卫,86,燕郊首尔甜城林荫大道,3000,2,,
